In [5]:
import pandas as pd
import json
import glob
import os
import numpy as np

# Veri yolları
PROCESSED_DIR = "../data/processed"

print("Kütüphaneler yüklendi.")

Kütüphaneler yüklendi.


In [6]:
all_records = []
json_files = sorted(glob.glob(os.path.join(PROCESSED_DIR, "*.json")))

print(f"Toplam {len(json_files)} dosya bulundu.")

for file_path in json_files:
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    year = data.get("year")

    # Her bir soruyu satır haline getir
    for q in data.get("questions", []):
        analysis = q.get("analysis", {})

        # Zorluk derecesini temizle
        raw_difficulty = analysis.get("tahmini_zorluk", 0)
        try:
            difficulty = int(str(raw_difficulty).split()[0])
        except (ValueError, IndexError):
            difficulty = 0

        # --- ŞIKLARI AYIKLAMA ---
        options_list = q.get("options", [])

        # Liste boşsa veya eksikse hata vermemesi için güvenli atama yapıyoruz
        opt_a = options_list[0] if len(options_list) > 0 else ""
        opt_b = options_list[1] if len(options_list) > 1 else ""
        opt_c = options_list[2] if len(options_list) > 2 else ""
        opt_d = options_list[3] if len(options_list) > 3 else ""

        record = {
            "Yil": year,
            "Soru_ID": q.get("id"),
            "Ana_Konu": analysis.get("ana_konu", "Bilinmiyor"),
            "Alt_Konu": analysis.get("alt_konu", "Belirtilmemis"),
            "Zorluk": difficulty,
            "Soru_Metni": q.get("text"),
            # Şıklar buraya ekleniyor
            "Secenek_A": opt_a,
            "Secenek_B": opt_b,
            "Secenek_C": opt_c,
            "Secenek_D": opt_d,
            "Dogru_Cevap": q.get("answer")
        }
        all_records.append(record)

print(f"Toplam {len(all_records)} soru işlendi ve listeye eklendi.")

Toplam 8 dosya bulundu.
Toplam 160 soru işlendi ve listeye eklendi.


In [7]:
# DataFrame oluştur
df = pd.DataFrame(all_records)

# Veri tiplerini düzenle
df["Yil"] = df["Yil"].astype(int)
df["Soru_ID"] = df["Soru_ID"].astype(int)

# Sütun sırasını en mantıklı hale getirelim
cols = ["Yil", "Soru_ID", "Ana_Konu", "Alt_Konu", "Zorluk",
        "Soru_Metni", "Secenek_A", "Secenek_B", "Secenek_C", "Secenek_D", "Dogru_Cevap"]
df = df[cols]

# CSV olarak kaydet (İstersen Excel olarak da kaydedebilirsin)
df.to_csv("../data/lgs_full_data.csv", index=False, encoding='utf-8-sig')

print(f"✅ Tablo oluşturuldu! Toplam Soru: {len(df)}")

# Tabloyu göster
# Pandas ayarları: Metinler kesilmesin diye sütun genişliğini artırıyoruz
pd.set_option('display.max_colwidth', 50)
display(df.head())

# Tablo hakkında bilgi
print("\n--- Veri Seti Özeti ---")
print(df.info())

✅ Tablo oluşturuldu! Toplam Soru: 160


,Yil,Soru_ID,Ana_Konu,Alt_Konu,Zorluk,Soru_Metni,Secenek_A,Secenek_B,Secenek_C,Secenek_D,Dogru_Cevap
0,2018,1,Paragrafta Anlam,Ana Düşünce,3,"Okuru olmayan bir ülkede dergi çıkartmak, suya...",A) büyük çaba gerektirmek - satış amacı gütmemek,B) kalıcı olmaya çalışmak - ilgi çekmemek,C) boş yere uğraşmak - yeterince tercih edilmemek,D) sıkıntılara göğüs germek - köklü bir geçmiş...,C
1,2018,2,Dil Bilgisi,Anlatım Bozukluğu,3,"Eğer yere uzanmış, hareketsiz, sessiz bir geyi...","A) ""yakınlarda olmalı"" sözü yerine ""yakınlarda...","B) ""endişelenmeniz"" kelimesi yerine ""endişe du...","C) ""sessiz"" kelimesi cümleden atılmalı.","D) ""uzanmış"" kelimesi yerine ""yatmış"" kelimesi...",A
2,2018,3,Paragrafta Anlam,Ana Düşünce,3,Son yıllarda toplumumuzun görsel sanatlar sevg...,A) Çevresindekilerle iletişime kapalı olmak,B) Sanattan yoksun bir hayat sürdürmek,C) Hayata farklı açılardan bakamamak,D) Tek bir sanat dalına yoğunlaşmak,B
3,2018,4,Paragrafta Anlam,Koşul Anlamı,3,"(I) Japonya, uzay çöplerini temizlemek için ür...",A) I.,B) II.,C) III.,D) IV.,D
4,2018,5,Paragrafta Anlam,Ana Düşünce,3,Teknolojik cihazları hayatımızın her sahasında...,"A) Yazar, düşüncelerini sorular yoluyla anlatm...",B) Karşılaştırmadan yararlanılmıştır.,C) Öznel ifadelere yer verilmiştir.,"D) Düşünce, sayısal verilerle kanıtlanmaya çal...",A



--- Veri Seti Özeti ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160 entries, 0 to 159
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Yil          160 non-null    int64 
 1   Soru_ID      160 non-null    int64 
 2   Ana_Konu     160 non-null    object
 3   Alt_Konu     160 non-null    object
 4   Zorluk       160 non-null    int64 
 5   Soru_Metni   160 non-null    object
 6   Secenek_A    160 non-null    object
 7   Secenek_B    160 non-null    object
 8   Secenek_C    160 non-null    object
 9   Secenek_D    160 non-null    object
 10  Dogru_Cevap  160 non-null    object
dtypes: int64(3), object(8)
memory usage: 13.9+ KB
None


In [8]:
# Hangi yıl kaç soru var?
print("\n--- Yıllara Göre Soru Sayıları ---")
print(df["Yil"].value_counts().sort_index())

# Hangi ana konudan toplam kaç soru var?
print("\n--- Konulara Göre Toplam Soru Sayıları ---")
print(df["Ana_Konu"].value_counts())

# Veriyi CSV olarak kaydedelim (Daha sonra hızlı yüklemek için)
output_csv_path = "../data/lgs_processed_all_years.csv"
df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')

print(f"\n✅ Veri seti başarıyla '{output_csv_path}' dosyasına kaydedildi.")


--- Yıllara Göre Soru Sayıları ---
Yil
2018    20
2019    20
2020    20
2021    20
2022    20
2023    20
2024    20
2025    20
Name: count, dtype: int64

--- Konulara Göre Toplam Soru Sayıları ---
Ana_Konu
Paragrafta Anlam    77
Sözel Mantık        27
Dil Bilgisi         17
Yazım Kuralları     13
Sözcükte Anlam      13
Grafik Yorumlama    10
Fiilimsiler          1
Cümle Türleri        1
Söz Sanatları        1
Name: count, dtype: int64

✅ Veri seti başarıyla '../data/lgs_processed_all_years.csv' dosyasına kaydedildi.
